In [ ]:
import os

import torch
from utils.net import get_resnet50
from utils.dataset import get_dataloaders
from utils.loop import loop
from utils.config import Config
from pathlib import Path

In [ ]:
def make_experiment(folder_real: str, folders_fake: list[str] = None, folders_both: list[str] = None, folders_train: list[str] = None, folders_valid: list[str] = None, base_path: str = None):
    learning_rate = 1e-4
    batch_size = 64
    n_epochs = 5
    train_loader, valid_loader, test_loader = get_dataloaders(folder_real, folders_fake, folders_both, folders_train, folders_valid, batch_size)

    model = get_resnet50()
    model_weights = "ResNet50_Weights.IMAGENET1K_V2"

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_function = torch.nn.functional.binary_cross_entropy_with_logits

    T = ("T_" + "_".join([Path(f).stem for f in folders_train])) if folders_train else ""
    V = "V_" + "_".join([Path(f).stem for f in folders_valid]) if folders_valid else ""
    B = "B_" + "_".join([Path(f).stem for f in folders_both]) if folders_both else ""
    name = f"{T}_{V}_{B}"

    save_path = os.path.join(base_path, name)
    if not os.path.exists(save_path):
        os.mkdir(save_path)

    C = {
        "name": name,
        "learning_rate": learning_rate,
        "n_epochs": n_epochs,
        "batch_size": batch_size,
        "folder_real": folder_real,
        "folders_test": folders_valid,
        "folders_both": folders_both,
        "folders_train": folders_train,
        "folders_valid": folders_valid,
        "model_type": model._get_name(),
        "model_weights": model_weights,
        "loss_function": loss_function,
        "optimizer": optimizer.__class__
    }


    config = Config(C)
    config.save(os.path.join(save_path, name))
    loop(model, optimizer, loss_function, save_path, train_loader, valid_loader, test_loader, n_epochs)

## Leave-one-out

In [ ]:
folder_real = "faces/real/"
fake_base = "faces/fake/"
folders_fake = [os.path.join(fake_base, f) for f in os.listdir(fake_base)]
base_path = "experiments/leave_one_out/"
for i in range(0, folders_fake.__len__()):
    folders_train = folders_fake.copy()
    folders_valid = [folders_train.pop(i)]
    print(folders_train, folders_valid)
    make_experiment(folder_real=folder_real, folders_fake=folders_fake, folders_train=folders_train, folders_valid=folders_valid, base_path=base_path)
    

## Same train and valid

In [ ]:
folder_real = "faces/real/"
folders_fake = os.listdir("faces/fake/")
base_path = "experiments/other/"
for f in folders_fake:
    if "FaceSwap" in f: continue
    folders_both = [os.path.join("faces/fake/", f)]
    ff = folders_both
    print(f)
    make_experiment(folder_real=folder_real, folders_fake=ff, folders_both=folders_both, base_path=base_path)